In [1]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 1.1.0 cuda: True


In [5]:
## Load dataset
data_path = "../gmm_dataset_c20k"
Data = torch.from_numpy(np.load(data_path + '/obs.npy')).float()

NUM_DATASETS, N, D = Data.shape
K = 3 ## number of clusters
SAMPLE_SIZE = 10
NUM_HIDDEN_LOCAL = 32

MCMC_SIZE = 10
BATCH_SIZE = 500
NUM_EPOCHS = 250
LEARNING_RATE = 1e-4
CUDA = torch.cuda.is_available()
PATH = 'AG-hard-os-%dsteps-%dsamples' % (MCMC_SIZE, SAMPLE_SIZE)
DEVICE = torch.device('cuda:0')

Train_Params = (NUM_EPOCHS, NUM_DATASETS, SAMPLE_SIZE, BATCH_SIZE, CUDA, DEVICE, PATH)
Model_Params = (N, K, D, MCMC_SIZE, PRIOR_FLAG, ONLY_FORWARD)

In [6]:
from local_enc import *
from global_oneshot import *
from global_enc_v1 import *
## if reparameterize continuous variables
Reparameterized = False
# initialization
enc_z = Enc_z(K, D, NUM_HIDDEN_LOCAL, CUDA, DEVICE)
enc_eta = Enc_eta(K, D, CUDA, DEVICE, Reparameterized)
oneshot_eta = Oneshot_eta(K, D, CUDA, DEVICE, Reparameterized)
if CUDA:
    enc_z.cuda().to(DEVICE)
    enc_eta.cuda().to(DEVICE)
    oneshot_eta.cuda().to(DEVICE)
optimizer =  torch.optim.Adam(list(oneshot_eta.parameters())+list(enc_eta.parameters())+list(enc_z.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))
models = (oneshot_eta, enc_eta, enc_z)

In [7]:
enc_z.load_state_dict(torch.load("../weights/enc-z-%s" % PATH))
enc_eta.load_state_dict(torch.load("../weights/enc-eta-%s" % PATH))
oneshot_eta.load_state_dict(torch.load("../weights/oneshot-eta-%s" % PATH))

RuntimeError: Error(s) in loading state_dict for Enc_eta:
	Missing key(s) in state_dict: "gamma.0.weight", "gamma.0.bias". 

In [ ]:
MAX_MCMC_STEPS = 10 ## 12 is maximum mcmc steps
SAMPLE_SIZE = 1
BATCH_SIZE = 5
Vis_Interval = 2
##
colors = ['#0077BB', '#009988', '#EE7733']
gs = gridspec.GridSpec(BATCH_SIZE, 2+int(MAX_MCMC_STEPS / Vis_Interval))
gs.update(left=0.0 , bottom=0.0, right=1.0, top=1.0, wspace=0, hspace=0)
fig = plt.figure(figsize=(30,25))

indices = torch.arange(NUM_DATASETS)
step = 2
batch_indices = indices[step*BATCH_SIZE : (step+1)*BATCH_SIZE]
obs = Data[batch_indices]
obs = shuffler(obs).repeat(SAMPLE_SIZE, 1, 1, 1)
if CUDA:
    obs =obs.cuda().to(DEVICE)
    
xs = obs[0].cpu().data.numpy()    
for b in range(BATCH_SIZE):
    xb = xs[b]
    ax = fig.add_subplot(gs[b, 0])
    ax.scatter(xb[:, 0], xb[:, 1], c='k')
    ax.set_ylim([-12, 12])
    ax.set_xlim([-12, 12])
    ax.set_xticks([])
    ax.set_yticks([])
    if b == 0:
        ax.set_title('Data', fontsize=30)
        
obs_tau, obs_mu = enc_eta.sample_prior(SAMPLE_SIZE, BATCH_SIZE)
q_z, p_z = enc_z.forward(obs, obs_tau, obs_mu, N, K, SAMPLE_SIZE, BATCH_SIZE)
state = q_z['zs'].value ## S * B * N * K    

E_z = state.mean(0).cpu().data.numpy()
E_mu = obs_mu.mean(0).cpu().data.numpy()
E_tau = obs_tau.mean(0).cpu().data.numpy()


# q_eta, p_eta, q_nu = oneshot_eta(obs, K, D)
# obs_mu = q_eta['means'].value
# obs_tau = q_eta['precisions'].value
# q_z, p_z = enc_z.forward(obs, obs_tau, obs_mu, N, K, SAMPLE_SIZE, BATCH_SIZE)
# state = q_z['zs'].value ## S * B * N * K
# E_z = q_z['zs'].dist.probs[0].cpu().data.numpy()
# E_mu = q_eta['means'].dist.loc[0].cpu().data.numpy()
# E_tau = (q_eta['precisions'].dist.concentration[0] / q_eta['precisions'].dist.rate[0]).cpu().data.numpy()

for b in range(BATCH_SIZE):
    ax = fig.add_subplot(gs[b, 1])
    xb = xs[b]
    zb = E_z[b]
    mu = E_mu[b].reshape(K, D)
    sigma2 = 1. / E_tau[b]
    assignments = zb.argmax(-1)
    for k in range(K):
        cov_k = np.diag(sigma2[k])
        xk = xb[np.where(assignments == k)]
        ax.scatter(xk[:, 0], xk[:, 1], c=colors[k], zorder=3)
        plot_cov_ellipse(cov=cov_k, pos=mu[k], nstd=2, ax=ax, alpha=0.3, color=colors[k])
    ax.set_ylim([-12, 12])
    ax.set_xlim([-12, 12])
    ax.set_xticks([])
    ax.set_yticks([])
    if b == 0:
        ax.set_title("Step 0", fontsize=30)

for m in range(MAX_MCMC_STEPS):
    q_eta, p_eta, q_nu = enc_eta(obs, state, K, D)
    obs_tau, obs_mu, log_w_eta_f, log_w_eta_b  = Incremental_eta(q_eta, p_eta, obs, state, K, D, obs_tau, obs_mu)
    symkl_detailed_balance_eta, eubo_p_q_eta, w_sym_eta, w_f_eta = detailed_balances(log_w_eta_f, log_w_eta_b)
    obs_mu, obs_tau = resample_eta(obs_mu, obs_tau, w_f_eta, idw_flag=True) ## resample eta
    q_z, p_z = enc_z.forward(obs, obs_tau, obs_mu, N, K, SAMPLE_SIZE, BATCH_SIZE)
    state, log_w_z_f, log_w_z_b = Incremental_z(q_z, p_z, obs, obs_tau, obs_mu, K, D, state)
    if (m+1) % Vis_Interval == 0:
        E_z = q_z['zs'].dist.probs[0].cpu().data.numpy()
        E_mu = q_eta['means'].dist.loc[0].cpu().data.numpy()
        E_tau = (q_eta['precisions'].dist.concentration[0] / q_eta['precisions'].dist.rate[0]).cpu().data.numpy()
        for b in range(BATCH_SIZE):
            ax = fig.add_subplot(gs[b, 1+int((m+1)/Vis_Interval)])
            xb = xs[b]
            zb = E_z[b]
            mu = E_mu[b].reshape(K, D)
            sigma2 = 1. / E_tau[b]
            assignments = zb.argmax(-1)
            for k in range(K):
                cov_k = np.diag(sigma2[k])
                xk = xb[np.where(assignments == k)]
                ax.scatter(xk[:, 0], xk[:, 1], c=colors[k])
                plot_cov_ellipse(cov=cov_k, pos=mu[k], nstd=2, ax=ax, alpha=0.2, color=colors[k])
            ax.set_ylim([-12, 12])
            ax.set_xlim([-12, 12])
            ax.set_xticks([])
            ax.set_yticks([])
            if b == 0:
                ax.set_title('Step %d' % (m+1), fontsize=30)
plt.savefig('../results/sample-gmm-' + PATH + '.svg')

In [16]:
Train_Params_Test = (NUM_DATASETS, SAMPLE_SIZE, BATCH_SIZE, CUDA, DEVICE, PATH)
Model_Params = (N, K, D, MCMC_SIZE)
def test_propagation(models, objective, data, Model_Params, Train_Params):
    """
    generic training function
    """
#     KLs = {"kl_eta_ex" : [],"kl_eta_in" : [],"kl_z_ex" : [],"kl_z_in" : []}
    Metrics = {"symKL_DB_eta" : [], "symKL_DB_z" : [], "ess" : []}
    (NUM_DATASETS, S, B, CUDA, device, path) = Train_Params

    NUM_BATCHES = int((NUM_DATASETS / B))

    SubTrain_Params = (device, S, B) + Model_Params
    indices = torch.randperm(NUM_DATASETS)
    time_start = time.time()
    for step in range(NUM_BATCHES):
        batch_indices = indices[step*B : (step+1)*B]
        obs = data[batch_indices]
        obs = shuffler(obs).repeat(S, 1, 1, 1)
        if CUDA:
            obs =obs.cuda().to(device)
        metric_step = objective(models, obs, SubTrain_Params)
        ## gradient step

        for key in Metrics.keys():
            if Metrics[key] == None:
                Metrics[key] = [metric_step[key].cpu().data.numpy()]
            else:
                Metrics[key].append(metric_step[key].cpu().data.numpy())
                
#         if step % 100 == 0:
#             time_end = time.time()
#             print('iteration:%d/%d' % (step, NUM_BATCHES))
#             time_start = time.time()
    return Metrics

def EUBO_init_eta_prior_test(models, obs, SubTrain_Params):
    """
    NO Resampling
    Learn neural gibbs samplers for both eta and z,
    non-reparameterized-style gradient estimation
    initialize eta
    """
    (device, sample_size, batch_size, N, K, D, mcmc_size) = SubTrain_Params
    esss = torch.zeros(mcmc_size+1).cuda().to(device)
    symkls_DB_eta = torch.zeros(mcmc_size+1).cuda().to(device)
    symkls_DB_z = torch.zeros(mcmc_size+1).cuda().to(device)
    (enc_eta, enc_z) = models
    obs_tau, obs_mu = enc_eta.sample_prior(sample_size, batch_size)
    q_z, p_z = enc_z.forward(obs, obs_tau, obs_mu, N, K, sample_size, batch_size)
    log_p_z = p_z['zs'].log_prob
    log_q_z = q_z['zs'].log_prob
    state = q_z['zs'].value ## S * B * N * K
    log_obs_n = Log_likelihood(obs, state, obs_tau, obs_mu, K, D, cluster_flag=False)
    log_w_f_z = log_obs_n + log_p_z - log_q_z
    w_f_z = F.softmax(log_w_f_z, 0).detach()

    symkls_DB_eta[0] = (w_f_z * log_w_f_z).sum(0).sum(-1).mean() - log_w_f_z.sum(-1).mean()
    symkls_DB_z[0] = symkls_DB_eta[0] ##
    esss[0] = (1. / (w_f_z**2).sum(0)).mean()
    for m in range(mcmc_size):
#         state = resample_state(state, w_f_z, idw_flag=True)
        q_eta, p_eta, q_nu = enc_eta(obs, state, K, D)
        obs_tau, obs_mu, log_w_eta_f, log_w_eta_b  = Incremental_eta(q_eta, p_eta, obs, state, K, D, obs_tau, obs_mu)
        symkl_detailed_balance_eta, eubo_p_q_eta, w_sym_eta, w_f_eta = detailed_balances(log_w_eta_f, log_w_eta_b)
        obs_mu, obs_tau = resample_eta(obs_mu, obs_tau, w_f_eta, idw_flag=True) ## resample eta
        q_z, p_z = enc_z.forward(obs, obs_tau, obs_mu, N, K, sample_size, batch_size)
        state, log_w_z_f, log_w_z_b = Incremental_z(q_z, p_z, obs, obs_tau, obs_mu, K, D, state)
        symkl_detailed_balance_z, eubo_p_q_z, w_sym_z, w_f_z = detailed_balances(log_w_z_f, log_w_z_b)
        ## symmetric KLs as metrics
        symkls_DB_eta[m+1] = symkl_detailed_balance_eta
        symkls_DB_z[m+1] = symkl_detailed_balance_z
        esss[m+1] = ((1. / (w_sym_eta**2).sum(0)).mean() + (1. / (w_sym_z**2).sum(0)).mean() ) / 2
    metric_step = {"symKL_DB_eta" : symkls_DB_eta, "symKL_DB_z" : symkls_DB_z, "ess" : esss}
    return metric_step

In [18]:
ESSs = []
DB_ETAs = []
DB_Zs = []
for i in range(1000):
    metrics = test_propagation(models, EUBO_init_eta_prior_test, Data, Model_Params, Train_Params_Test)
    DB_ETAs.append(np.array(metrics['symKL_DB_eta']).mean(0))
    DB_Zs.append(np.array(metrics['symKL_DB_z']).mean(0))
    ESSs.append(np.array(metrics['ess']).mean(0))
    print('iteration : %d' % i)

iteration : 0
iteration : 1
iteration : 2
iteration : 3
iteration : 4
iteration : 5
iteration : 6
iteration : 7
iteration : 8
iteration : 9
iteration : 10
iteration : 11
iteration : 12
iteration : 13
iteration : 14
iteration : 15
iteration : 16
iteration : 17
iteration : 18
iteration : 19
iteration : 20
iteration : 21
iteration : 22
iteration : 23
iteration : 24
iteration : 25
iteration : 26
iteration : 27
iteration : 28
iteration : 29
iteration : 30
iteration : 31
iteration : 32
iteration : 33
iteration : 34
iteration : 35
iteration : 36
iteration : 37
iteration : 38
iteration : 39
iteration : 40
iteration : 41
iteration : 42
iteration : 43
iteration : 44
iteration : 45
iteration : 46
iteration : 47
iteration : 48
iteration : 49
iteration : 50
iteration : 51
iteration : 52
iteration : 53
iteration : 54
iteration : 55
iteration : 56
iteration : 57
iteration : 58
iteration : 59
iteration : 60
iteration : 61
iteration : 62
iteration : 63
iteration : 64
iteration : 65
iteration : 66
itera

KeyboardInterrupt: 

In [11]:
DB_eta = np.array(metrics['symKL_DB_eta']).mean(0)
DB_z = np.array(metrics['symKL_DB_z']).mean(0)
ess =  np.array(metrics['ess']).mean(0)


(200, 11)

In [19]:
ess

NameError: name 'ess' is not defined